In [1]:
import numpy as np
import pandas as pd
import pyomo.environ as pyo
from measure_optimize import MeasurementOptimizer, DataProcess, covariance_lib
import matplotlib.pyplot as plt
import pickle 

## Input Setting

### All measurements and index

- ads.gas_inlet.F (0)
- ads.gas_outlet.F (1)
- ads.gas_outlet.T (2)
- ads.gas_outlet.z (3)
- des.gas_inlet.F (4)
- des.gas_outlet.F (5)
- des.gas_outlet.T (6)
- des.gas_outlet.z (7)
- ads.T19 (8)
- ads.T23 (9)
- ads.T28 (10)
- ads.z19 (11)
- ads.z23 (12)
- ads.z28 (13)

One-time cost ones: 0,1,2, 4,5,6, 8,9,10

Time-cost ones: 3,7,11,12,13

Names in str: 'Ads.gas_inlet.F', 'Ads.gas_outlet.F', 'Ads.gas_outlet.T', 
             'Ads.gas_outlet.z("CO2")', 'Des.gas_inlet.F', 'Des.gas_outlet.F', 
             'Des.gas_outlet.T', 'Des.gas_outlet.z("CO2")', 'Ads.T_g.Value(19,10)', 
             'Ads.T_g.Value(23,10)', 'Ads.T_g.Value(28,10)', 'Ads.z("CO2",19,10)', 
             'Ads.z("CO2",23,10)', 'Ads.z("CO2",28,10)'

In [2]:
Nt =110

max_manual_num = 5 
min_interval_num = 10

In [3]:
static_ind = [0,1,2,3,4,5,6,7,8,9,10]
dynamic_ind = [11,12,13,14,15]
all_ind = static_ind+dynamic_ind

num_total = len(all_ind)

In [4]:
all_names_strategy3 = ['Ads.gas_inlet.F', 'Ads.gas_outlet.F', 'Ads.gas_outlet.T', 
             'Des.gas_inlet.F', 'Des.gas_outlet.F', 
             'Des.gas_outlet.T',  'Ads.T_g.Value(19,10)', 
             'Ads.T_g.Value(23,10)', 'Ads.T_g.Value(28,10)', # all static
            'Ads.gas_outlet.z("CO2").static', 'Des.gas_outlet.z("CO2").static', # static z 
            'Ads.gas_outlet.z("CO2").dynamic', 'Des.gas_outlet.z("CO2").dynamic', # dynamic z 
            'Ads.z("CO2",19,10)', 'Ads.z("CO2",23,10)', 'Ads.z("CO2",28,10)']

In [5]:

static_cost = [1000, #ads.gas_inlet.F (0)
    1000, #ads.gas_outlet.F (1)
     500, #ads.gas_outlet.T (2)
    1000, #des.gas_inlet.F (4)
    1000, #des.gas_outlet.F (5)
     500, #des.gas_outlet.T (6)
     1000, #ads.T19 (8)
     1000, #ads.T23 (9)
     1000, #ads.T28 (10)
    7000,
    7000]


static_cost.extend([100, 100, 500, 500, 500])

dynamic_cost = [0]*len(static_ind)
dynamic_cost.extend([100]*len(dynamic_ind))

max_manual = [max_manual_num]*num_total
min_time_interval = [min_interval_num]*num_total

In [6]:
print("if selected all static:", sum(static_cost))
print("if selected all static without two z:", sum(static_cost)-7000*2)

if selected all static: 23700
if selected all static without two z: 9700


In [7]:
measure_info = pd.DataFrame({
    "name": all_names_strategy3,
    "Q_index": all_ind,
        "static_cost": static_cost,
    "dynamic_cost": dynamic_cost,
    "min_time_interval": min_time_interval, 
    "max_manual_number": max_manual
})

print(measure_info)

                               name  Q_index  static_cost  dynamic_cost  \
0                   Ads.gas_inlet.F        0         1000             0   
1                  Ads.gas_outlet.F        1         1000             0   
2                  Ads.gas_outlet.T        2          500             0   
3                   Des.gas_inlet.F        3         1000             0   
4                  Des.gas_outlet.F        4         1000             0   
5                  Des.gas_outlet.T        5          500             0   
6              Ads.T_g.Value(19,10)        6         1000             0   
7              Ads.T_g.Value(23,10)        7         1000             0   
8              Ads.T_g.Value(28,10)        8         1000             0   
9    Ads.gas_outlet.z("CO2").static        9         7000             0   
10   Des.gas_outlet.z("CO2").static       10         7000             0   
11  Ads.gas_outlet.z("CO2").dynamic       11          100           100   
12  Des.gas_outlet.z("CO2

In [8]:
print(measure_info[measure_info['name']=='Ads.gas_outlet.z("CO2").static'].index.values)
print(len(measure_info))

[9]
16


In [10]:
dataObject = DataProcess()
dataObject.read_jaco('./RotaryBed/Q'+str(Nt)+'_scale.csv')
Q = dataObject.get_Q_list([0,1,2,4,5,6,8,9,10,3,7],
                        [3,7,11,12,13], Nt)


In [11]:
calculator = MeasurementOptimizer(Q, measure_info, error_opt=covariance_lib.identity, verbose=True)

fim_expect = calculator.fim_computation()

print(np.shape(calculator.fim_collection))

(314721, 5, 5)


### Debug functions: check each unit FIM

In [ ]:
print(calculator.head_pos_dynamic_flatten)
print(calculator.static_idx)
print(calculator.dynamic_idx)
print(calculator.Q_dynamic_flatten[451][0])

In [ ]:
# check Q is rightly arranged 
for i in range(16):
    print(Q[i][0])

In [ ]:
def check(fim_set):
    for i in range(num_total):
        for j in range(num_total):
            fim = fim_set[i*num_total+j]
            if np.any(fim):
                logdet = np.log(np.linalg.det(fim))
                if logdet < 0:
                    print("unit FIM with small Det: ", i, j, ', logdet:', logdet)
                if max(np.linalg.eigvals(fim))/min(np.linalg.eigvals(fim)) > 1000:
                    print("unit FIM with big condition number:", i, j, ", eigvals:", np.linalg.eigvals(fim))
                    
def print_fim(t):
    fim = calculator.fim_collection[t*num_total+t]
    print("Measurement name: ", all_names_strategy3[t])
    #print(fim)
    print("Trace:", np.trace(fim))

In [ ]:
print(num_total)
for i in range(11):
    print_fim(i)

In [ ]:


total_fim = np.zeros((5,5))
for i in range(314721):
    total_fim += calculator.fim_collection[i]
print(total_fim)

### Initialize

In [12]:
num_static = len(static_ind)
num_dynamic  = len(dynamic_ind)
num_total = num_static + num_dynamic*Nt

In [13]:
#with open('MarResults-A-strategy3/Mar20_A_110_8000_strategy3_mip', 'rb') as f:
with open('MarResult-strategy3-Dopt/Mar31_D_110_27700', 'rb') as f:
    x = pickle.load(f)
    
print(x)

init_cov_y = x.copy()

for i in range(11):
    for j in range(11):
        init_cov_y[i,j] = 1
    
# initialize all decisions
#init_cov_y = np.ones(x.shape) 

[[1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [14]:
for i in range(11,231):
    for j in range(11,231):
        init_cov_y[i,j] = 0

In [15]:
#with open('strategy3_all_covy', 'rb') as f:
with open('MarResult-strategy3-Dopt/Mar31_FIM_D_110_27700', 'rb') as f:
    fim_prior = pickle.load(f)
    
print(fim_prior)

[[ 5.55260011e+03 -2.61972445e+00 -4.14130933e+03  1.09879444e+04
  -5.00376769e+02]
 [-2.61972445e+00  5.22032483e+00  7.68731822e+00  5.03877570e+01
  -1.05219552e+01]
 [-4.14130933e+03  7.68731822e+00  3.32562801e+03 -8.52086707e+03
   1.66399892e+02]
 [ 1.09879444e+04  5.03877570e+01 -8.52086707e+03  2.52492460e+04
  -8.96253626e+02]
 [-5.00376769e+02 -1.05219552e+01  1.66399892e+02 -8.96253626e+02
   3.78315460e+02]]


In [16]:

mip_option = True
objective = "A"
sparse_opt = True
fix_opt = False

manual_num = 20
budget_opt = 1000

num_dynamic_time = np.linspace(2,220,Nt)

static_dynamic = [[9,11], [10,12]]
#static_dynamic = None
time_interval_for_all = True

dynamic_time_dict = {}
for i, tim in enumerate(num_dynamic_time):
    dynamic_time_dict[i] = tim 

In [17]:
mod = calculator.continuous_optimization(mixed_integer=mip_option, 
                      obj=objective, 
                    fix=fix_opt, 
                    sparse=sparse_opt, 
                    num_dynamic_t_name = num_dynamic_time, 
                    manual_number = manual_num, 
                    budget=budget_opt,
                    init_cov_y= init_cov_y, 
                    initial_fim = fim_prior,
                    static_dynamic_pair=static_dynamic,
                    time_interval_all_dynamic = time_interval_for_all)

mod = calculator.solve(mod, mip_option=mip_option, objective = objective)

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Set parameter LogFile to value "/tmp/tmph6es34sx.log"
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Core(TM) i7-8850H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 471931 rows, 157663 columns and 1113997 nonzeros
Model fingerprint: 0x1efd2aca
Variable types: 17 continuous, 157646 integer (157646 binary)
Coefficient statistics:
  Matrix range     [2e-13, 2e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective -0.0000000
Presolve removed 9393 rows and 1138 columns (presolve time = 5s) ...
Presolve removed 9393 rows and 1138 columns (presolve time = 10s) ...
Presolve removed 50418 rows a

In [18]:
print(calculator.dynamic_install_cost)

[100, 100, 500, 500, 500]


In [19]:
fim_result = np.zeros((5,5))
for i in range(5):
    for j in range(i,5):
        fim_result[i,j] = fim_result[j,i] = pyo.value(mod.TotalFIM[i,j])
        
print(fim_result)  
print('trace:', np.trace(fim_result))
print('det:', np.linalg.det(fim_result))
print(np.linalg.eigvals(fim_result))

print("Pyomo OF:", pyo.value(mod.Obj))
print("Log_det:", np.log(np.linalg.det(fim_result)))

[[ 5.36204808e+03 -9.03127412e+00 -4.17278886e+03  1.07974958e+04
  -2.75923843e+02]
 [-9.03127412e+00  2.99606521e-01  8.08083904e+00 -1.15941640e+01
  -1.35622981e+00]
 [-4.17278886e+03  8.08083904e+00  3.25961288e+03 -8.34441347e+03
   2.00296313e+02]
 [ 1.07974958e+04 -1.15941640e+01 -8.34441347e+03  2.28129810e+04
  -5.61026805e+02]
 [-2.75923843e+02 -1.35622981e+00  2.00296313e+02 -5.61026805e+02
   4.65786229e+01]]
trace: 31481.520152683108
det: 78390965.84086484
[3.11097311e+04 3.33795612e+02 3.63837393e+01 1.46835475e+00
 1.41302813e-01]
Pyomo OF: 31481.520152683108
Log_det: 18.17721924705934


In [20]:
ans_y, sol_y = calculator.extract_solutions(mod)


Ads.gas_inlet.F :  0.0
Ads.gas_outlet.F :  0.0
Ads.gas_outlet.T :  1.0
Des.gas_inlet.F :  0.0
Des.gas_outlet.F :  0.0
Des.gas_outlet.T :  1.0
Ads.T_g.Value(19,10) :  0.0
Ads.T_g.Value(23,10) :  0.0
Ads.T_g.Value(28,10) :  0.0
Ads.gas_outlet.z("CO2").static :  0.0
Des.gas_outlet.z("CO2").static :  0.0
Ads.gas_outlet.z("CO2").dynamic
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Des.gas_outlet.z("CO2").dynamic
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.

In [21]:
for i in range(5):
    print(all_names_strategy3[i+11])
    for t in range(len(sol_y[0])):
        if sol_y[i][t] > 0.95:
            print(dynamic_time_dict[t])
print('pyomo calculated cost:', pyo.value(mod.cost))

for i in [11,12,13,14,15]:
    print(pyo.value(mod.if_install_dynamic[i]))

Ads.gas_outlet.z("CO2").dynamic
Des.gas_outlet.z("CO2").dynamic
Ads.z("CO2",19,10)
Ads.z("CO2",23,10)
Ads.z("CO2",28,10)
pyomo calculated cost: 1000.0
0.0
0.0
0.0
0.0
0.0


In [ ]:
# verify cost 
#real_cost = 0 
#for i in range(num_total):
#    real_cost += ans_y[i,i]*cost[i]
    
#dynamic_inst = []
#for j in range(num_static, num_dynamic+num_static):
#    dynamic_inst.append(pyo.value(mod.if_install_dynamic[j]))
#    for t in range(Nt):
#        if sol_y[j,t]>0.99:
#            dynamic_inst[j] = 1 
            
#real_cost += sum(dynamic_inst[i]*install_cost_dynamic[i] for i in range(num_dynamic))
#print(dynamic_inst)
#print('Real cost:', real_cost)


In [ ]:
count = 0 
for i in range(561):
    for j in range(i,561):
        #if pyo.value(mod.cov_y[i,j])<0.96:
        print(pyo.value(mod.cov_y[i,j]))
            #count += 1 
            
print(count)

In [ ]:
store = True

if store:
    file = open('Apr17_A_mip_1000', 'wb')

    pickle.dump(ans_y, file)

    file.close()
    
    file2 = open('Apr17_FIM_A_mip_1000', 'wb')

    pickle.dump(fim_result, file2)

    file2.close()